# Iterables, iterators, generators & range

Dnes se společně podíváme na to, co se vlastně děje na pozadí cyklu `for` a jak si můžeme iterování skrz prvky nějaké sekvence upravit dle libosti.

## Iterables (iterovatelné objekty)

Iterovatelné objekty jsou zjednodušeně řečeno všechny objekty v Pythonu, které se dají procházet prvek za prvkem a dají se tedy podstrčit cyklu `for`.

Jako n-tice

In [1]:
for cislo in 1, 2, 3:
    print(cislo)

1
2
3


seznamy

In [2]:
for cislo in [4, 5, 6]:
    print(cislo)

4
5
6


řetězce

In [3]:
for cislo in "789":
    print(cislo)

7
8
9


a mnohé další, třeba i nekonečné

In [4]:
from itertools import count

nasobky_peti = count(100, 5)
nasobky_peti

count(100, 5)

In [5]:
for x in nasobky_peti:
    if x > 150:
        break
    print(x)

100
105
110
115
120
125
130
135
140
145
150


## Iterátor

Na pozadí takového cyklu `for` vznikne z iterovatelného objektu iterátor. Iterátor je nový objekt, který má dva hlavní úkoly: 1. pamatovat si svůj aktuální stav a 2. na zavolání funkce `next` vrátit následující prvek. Iterátor se z iterovatelného objektu tvoří funkcí `iter` a `for` cyklus si s jeho pomocí můžeme implementovat i sami.

In [6]:
retezec = "pes"
iterator = iter(retezec)
iterator

In [7]:
next(iterator)

'p'

In [8]:
next(iterator)

'e'

In [9]:
next(iterator)

's'

Pokud se pokusíme v získávání dalších prvků pokračovat, dostaneme výjimku `StopIteration`, podle které cyklus `for` pozná, že je u konce.

In [10]:
next(iterator)

StopIteration: 

Protože si iterátor pamatuje svůj stav, nemůžeme jej projít vícekrát aniž bychom jej znovu nevytvořili.

## Reimplementace cyklu for

Když už tušíme, jaký je rozdíl mezi iterovatelým objektem a iterátorem, můžeme si zkusit implementovat vlastní cyklus `for`:

In [11]:
retezec = "kočka"

Takto by vypadal cyklus for procházející jednotlivé znaky z řetězce:

In [12]:
for znak in retezec:
    print(znak)

k
o
č
k
a


A takto se dá stejného efektu docílit pomocí cyklu `while`, funkcí `iter` a `next` a výjimky `StopIteration`:

In [13]:
iterator = iter(retezec)

while True:
    try:
        dalsi = next(iterator)
        print(dalsi)
    except StopIteration:
        break

k
o
č
k
a


## Funkce standardní knihovny vracející iterátory

In [14]:
en = enumerate(['a', 'b', 'c'])
en

In [15]:
next(en)

(0, 'a')

In [16]:
next(en)

(1, 'b')

Funkce jako `zip`, `reversed` a mnoho dalších fungují podobně a vracejí iterátor připravený k použití.

## Vlastní iterátor

Objekt, který má mít vlastnosti iterátorů musí implementovat metodu `__iter__`, která jej inicializuje jako iterátor a metodu `__next__`, která se postará o navrácení dalšího prvku a vyvolání výjimky na konci.

Jednoduchá třída pro iterátor vracející mocniny dvojky do určitého maxima se dá napsat takto:

In [17]:
class MocninyDvojky():
    def __init__(self, maximum):
        self.maximum = maximum
        self.exponent = 1

    def __iter__(self):
        self.exponent = 0
        return self

    def __next__(self):
        self.exponent += 1
        vysledek = 2 ** self.exponent
        if vysledek > self.maximum:
            raise StopIteration
        return vysledek

In [18]:
iterator = MocninyDvojky(50)
iterator

In [19]:
next(iterator)

4

In [20]:
next(iterator)

8

In [21]:
next(iterator)

16

In [22]:
next(iterator)

32

In [23]:
next(iterator)

StopIteration: 

Došli jsme na konec iterátoru a dostali jsme očekávanou výjimku. Když ovšem stejný iterátor použijeme ve `for` cyklu, bude fungovat. Jak je to možné? Je to proto, že `for` cyklus zavolá na iterátoru funkci `iter`, která zavolá metodu `__iter__`, která iterátor spustí od začátku.

In [24]:
for x in iterator:
    print(x)

2
4
8
16
32


Na takovou jednoduchou úlohu je toho kódu ale nějak moc. Pojďme se podívat, jak to samé udělat pomocí funkce. Iterátoru definovanému pomocí funkce nebo výrazu se říká generátor. Generátory jsou podmnožinou iterátorů a fungují velmi podobně. Jen mají tu výhodu, že nemusíme vše implementovat manuálně.

### Funkce

Funkcí definovaný generátor používá místo příkazu `return` příkaz `yield`. Jakmile se ve funkci dojde k `yield`, funkce se zastaví a vrátí hodnotu za `yield`. Při dalším zavolání funkce se pak spustí tam, kde předtím skončila.

In [25]:
def mocniny_dvojky(maximum):
    exponent = 0
    while True:
        exponent += 1
        vysledek = 2 ** exponent
        if vysledek > maximum:
            break
        yield vysledek

In [26]:
generator = mocniny_dvojky(50)
generator

<generator object mocniny_dvojky at 0x055F9D10>

In [27]:
next(generator)

2

In [28]:
next(generator)

4

In [29]:
next(generator)

8

In [30]:
next(generator)

16

In [31]:
next(generator)

32

In [32]:
next(generator)

StopIteration: 

Vyvolání `StopIteration` je v tomto případě zcela automatické a stane se tak, když je generátor ukončen pomocí `return`. To se v našem případě stane, když ukončíme `while` cyklus a ve funkci už není kam pokračovat.

### Výraz

Výrazem definovaný generátor je velmi podobný list comprehensions, ale má kolem sebe kulaté závorky. Je méně univerzální než funkce nebo třída, ale pro spoustu případů bohatě postačí.

In [33]:
generator = (2**n for n in range(1, 6))

In [34]:
next(generator)

2

In [35]:
next(generator)

4

In [36]:
next(generator)

8

In [37]:
next(generator)

16

In [38]:
next(generator)

32

## Výhody iterátorů

Hlavní výhodou iterátorů je to, že nemáme v paměti programu všechny prvky sekvence najednou. Jsou případy, kdy to není možné a pro výpočet potřebujeme znát celou sekvenci najednou, ale pak jsou také případy, kdy by se nám celá sekvence ani nevešla do paměti a iterátor je jediné možné řešení.

Například budeme chtít získat součet řady mocnin dvojky a zkusíme pro to použít seznam a generátor. Definice je v tomto případě velmi podobná.

In [39]:
seznam_mocnin = [2**n for n in range(10000)]

In [40]:
generator = (2**n for n in range(10000))

Ale velikost, které jednotlivé objekty zabírají v paměti, se velmi liší.

In [41]:
import sys

In [42]:
sys.getsizeof(seznam_mocnin)

43808

In [43]:
sys.getsizeof(generator)

56

A přitom pro sumu všech hodnot nepotřebujeme znát všechny prvky najednou, ale stačí nám jen přičítat do mezisoučtu postupně.

In [44]:
sum(seznam_mocnin) == sum(generator)

True

## Nevýhody

Hlavní nevýhodou je, že s iterátory nelze pracovat jako s existující sekvencí, takže se nemůžeme podívat na jeho délku:

In [45]:
iterator = iter("PyLadies")

In [46]:
len(iterator)

TypeError: object of type 'str_iterator' has no len()

Ani nemůžeme prozkoumat jednotlivé prvky:

In [47]:
iterator[2]

TypeError: 'str_iterator' object is not subscriptable

A když se pokusíme zjistit, zda je nějaký prvek uvnitř, dostaneme sice výsledek, ale za cenu vyčerpání části nebo celého iterátoru:

In [48]:
"a" in iterator

True

Tímto se iterátor částečně vyčerpal a další prvek, který z něj dostaneme, je "d".

In [49]:
next(iterator)

'd'

A protože další "a" už náš iterátor neobsahuje, vyčerpá se zcela a vrátí False.

In [50]:
"a" in iterator

False

In [51]:
next(iterator)

StopIteration: 

## Speciální případy

A pak jsou tady speciální případy sekvencí, které tak nějak vybočují z řady a implementují různé vlastnosti známých typů — jako třeba `range`.

`range` se dá projít cyklem `for` a dá se z něj vytvořit iterátor, takže je to zcela jistě iterovatelný objekt.

In [52]:
for x in range(3):
    print(x)

0
1
2


In [53]:
iterator = iter(range(3))

In [54]:
next(iterator)

0

In [55]:
next(iterator)

1

Ale samotný `range` iterátorem není.

In [56]:
next(range(3))

TypeError: 'range' object is not an iterator

Tím výčet zvláštností nekončí. `range` se stejně jako třeba seznam a na rozdíl od iterátorů průchodem nevyčerpá.

In [57]:
cisla = range(10)

In [58]:
[x**2 for x in cisla]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [59]:
[x**3 for x in cisla]

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]

Můžeme zjistit jeho délku.

In [60]:
len(cisla)

10

A bez obav se podívat, zda obsahuje nějaké prvky.

In [61]:
99 in cisla

False

In [62]:
99 in cisla

False

`range` je totiž tzv. lazy sekvence. To znamená, že má některé vlastnosti sekvencí (seznamů, n-tic, …), ale přitom si neuchovává všechny prvky v paměti a počítá je, až když jsou potřeba. „Lenost“ je společná vlastnost `range` a iterátorů.

In [63]:
cisla = range(10000)
seznam = list(cisla)

In [64]:
sys.getsizeof(cisla)

24

In [65]:
sys.getsizeof(seznam)

40028

## Závěrem

* Objekty, ze kterých lze vytvořit iterátor jsou tzv. iterables (iterovatelné).
* Iterátor si pamatuje, kde skončil, a dokáže dodávat další prvky (stará se o iteraci).
* Generátor je speciální případ iterátoru, který si můžeme sami definovat funkcí nebo výrazem.
* Správným použitím iterátorů si můžeme usnadnit práci a zajistit menší paměťovou náročnost programu.
* `range` není iterátor.

Pokud má někdo chuť jít hlouběji, Python obsahuje modul `collections.abc`, který obsahuje abstraktní třídy pro implementaci různých kolekcí. Více informací je k dispozici v [dokumentaci](https://docs.python.org/3/library/collections.abc.html). 